# Importing libraries

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

# !conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [23]:
!conda install -c anaconda beautifulsoup4
!conda install -c anaconda html5lib 

WARNING conda.base.context:use_only_tar_bz2(632): Conda is constrained to only using the old .tar.bz2 file format because you have conda-build installed, and it is <3.18.3.  Update or remove conda-build to get smaller downloads and faster extractions.
Solving environment: - ^C
failed

CondaError: KeyboardInterrupt

WARNING conda.base.context:use_only_tar_bz2(632): Conda is constrained to only using the old .tar.bz2 file format because you have conda-build installed, and it is <3.18.3.  Update or remove conda-build to get smaller downloads and faster extractions.
^C

CondaError: KeyboardInterrupt



In [2]:
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
# website_url

 # Reading the source code for a given web page and creating a BeautifulSoup (soup)object
 - __Prettify() function__ in BeautifulSoup will enable us to view how the tags are nested in the document.
 - If you carefully inspect the HTML script all the table contents i.e. names of the countries which we intend to extract is under class Wikitable Sortable.<br>

In [3]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(website_url,'lxml')
# print(soup.prettify())

- So our first task is to find class ‘wikitable sortable’ in the HTML script.
- Also note: Under table class ‘wikitable sortable’ all the entries for the table like postalcode/Borough name/Neighborhood are stored within < td > .... < /td >

In [4]:
toronto_tab = soup.find('table',{'class':'wikitable sortable'})
tds = toronto_tab.findAll('td')

stack = []
for tr in toronto_tab.findAll('tr'):
    for td in tr.findAll('td'):
        stack.append(td.text.strip())
# print(stack)

# Convert the scraped data into Pandas DataFrame to work in python.

In [5]:
Postcode_list=[]
Borough_list=[]
Neighborhood_list=[]
num_rows = len(stack)
num_rows = int(num_rows)

for i in range(0,num_rows,3):
    Postcode_list.append(stack[i])
    Borough_list.append(stack[i+1])
    Neighborhood_list.append(stack[i+2])

In [6]:
toronto_df=pd.DataFrame()
toronto_df['Postcode']=Postcode_list
toronto_df['Borough']=Borough_list
toronto_df['Neighborhood']=Neighborhood_list
print('The shape of toronto_df is {}'.format(toronto_df.shape))
toronto_df.head()

The shape of toronto_df is (288, 3)


,Postcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


# Clean the dataset as per the guidelines mentioned in the introduction.

In [7]:
toronto_df_NAN = toronto_df.replace('Not assigned', np.nan, regex=True)
toronto_df_NAN.head(3)

,Postcode,Borough,Neighborhood
0,M1A,NaN,NaN
1,M2A,NaN,NaN
2,M3A,North York,Parkwoods


- Removing all rows for which the Borough value is not assigned.

In [8]:
toronto_noNAN = toronto_df_NAN.dropna(subset = ['Borough'])
print('The size of toronto_noNAN is {}'.format(toronto_noNAN.shape))
toronto_df_noNAN = toronto_noNAN.reset_index(drop=True)
toronto_df_noNAN.head(7)

The size of toronto_noNAN is (211, 3)


,Postcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,NaN


- Putting Neighborhood value as Borough value if only neighborhood value was not assigned.

In [9]:
toronto_df_noNAN = toronto_df_noNAN.replace(np.nan, 'Not assigned', regex=True)

Borough_listN = toronto_df_noNAN['Borough'].tolist()
Neighborhood_listN = toronto_df_noNAN['Neighborhood'].tolist()

for j in range(0,len(Borough_listN),1):
    if(Neighborhood_listN[j]=='Not assigned'):
        Neighborhood_listN[j]=Borough_listN[j]
        
toronto_df_noNAN['Borough']=Borough_listN
toronto_df_noNAN['Neighborhood']=Neighborhood_listN

toronto_df_noNAN = toronto_df_noNAN.replace('Not assigned', np.nan, regex=True)
toronto_df_noNAN.isnull().values.any() # checks presence of np.nan in the entire dataset

False

In [10]:
toronto_df_noNAN.head(7)

,Postcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Queen's Park


- Combining rows when more than one neighborhood exists in one postal code area.

In [11]:
duplicate = toronto_df_noNAN.duplicated(subset=['Postcode'], keep='first') # assigns True to all the first, second .... duplicates of Postcode values
duplicate.head()

0    False
1    False
2    False
3     True
4    False
dtype: bool

In [12]:
toronto_cleanDF = toronto_df_noNAN

todrop_list = []
for k in range(0,len(Borough_listN),1):
    if(duplicate[k]):
        temp_var = Neighborhood_listN[k-1]
        temp_var = temp_var + ', ' + Neighborhood_listN[k]
        Neighborhood_listN[k] = temp_var
        todrop_list.append(k-1)
        
toronto_cleanDF['Neighborhood'] = Neighborhood_listN

In [13]:
print(todrop_list)

[2, 4, 8, 11, 13, 16, 17, 18, 19, 21, 22, 24, 29, 30, 31, 33, 34, 45, 46, 49, 50, 52, 55, 56, 58, 61, 62, 64, 66, 67, 70, 72, 74, 76, 77, 79, 80, 82, 85, 87, 89, 90, 93, 94, 96, 100, 102, 103, 104, 106, 108, 114, 117, 118, 122, 124, 127, 131, 132, 134, 137, 138, 139, 143, 145, 147, 148, 150, 152, 153, 155, 156, 157, 159, 160, 161, 162, 164, 165, 166, 167, 168, 169, 171, 172, 174, 175, 176, 177, 178, 179, 180, 185, 189, 191, 193, 194, 198, 199, 200, 201, 202, 203, 204, 206, 207, 208, 209]


In [14]:
toronto_DF = toronto_cleanDF.drop(todrop_list)
toronto_DF = toronto_DF.reset_index(drop=True)

In [15]:
toronto_DF.head()

,Postcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


- ## The final shape of the data set is:

In [16]:
toronto_DF.shape

(103, 3)

# Filtering it to Toronto and Scarborough

In [17]:
toronto_DF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103 entries, 0 to 102
Data columns (total 3 columns):
Postcode        103 non-null object
Borough         103 non-null object
Neighborhood    103 non-null object
dtypes: object(3)
memory usage: 2.5+ KB


In [18]:
toronto_DF.set_index(['Postcode','Borough'],inplace=True)
res = toronto_DF.groupby(level=['Postcode','Borough'], sort=False).agg( ','.join)
res.dtypes

Neighborhood    object
dtype: object

In [19]:
res.reset_index(inplace=True)

In [20]:
res.head()

,Postcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


In [21]:
res.columns

Index(['Postcode', 'Borough', 'Neighborhood'], dtype='object')

In [22]:
res['Borough'].unique()

array(['North York', 'Downtown Toronto', "Queen's Park", 'Etobicoke',
       'Scarborough', 'East York', 'York', 'East Toronto', 'West Toronto',
       'Central Toronto', 'Mississauga'], dtype=object)

In [23]:
tar_data = res[(res['Borough']=='Downtown Toronto') | 
               (res['Borough']=='East Toronto') | 
               (res['Borough']=='West Toronto')|
               (res['Borough']=='Central Toronto')|
              (res['Borough'] == 'Scarborough')]

In [24]:
tar_data.head()

,Postcode,Borough,Neighborhood
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
6,M1B,Scarborough,"Rouge, Malvern"
9,M5B,Downtown Toronto,"Ryerson, Garden District"
12,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
15,M5C,Downtown Toronto,St. James Town


In [25]:
tar_data.reset_index(inplace=True)

In [26]:
tar_data.drop('index',1,inplace=True)

/home/jupyterlab/conda/lib/python3.6/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [27]:
tar_data

,Postcode,Borough,Neighborhood
0,M5A,Downtown Toronto,"Harbourfront, Regent Park"
1,M1B,Scarborough,"Rouge, Malvern"
2,M5B,Downtown Toronto,"Ryerson, Garden District"
3,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
4,M5C,Downtown Toronto,St. James Town
5,M1E,Scarborough,"Guildwood, Morningside, West Hill"
6,M4E,East Toronto,The Beaches
7,M5E,Downtown Toronto,Berczy Park
8,M1G,Scarborough,Woburn
9,M5G,Downtown Toronto,Central Bay Street


# Segmenting and clustering of data

In [ ]:
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim 

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes 
import folium


Solving environment: \ 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - defaults/linux-64::anaconda==5.3.1=py37_0
  - defaults/linux-64::astropy==3.0.4=py37h14c3975_0
  - defaults/linux-64::bkcharts==0.2=py37_0
  - defaults/linux-64::blaze==0.11.3=py37_0
  - defaults/linux-64::bokeh==0.13.0=py37_0
  - defaults/linux-64::bottleneck==1.2.1=py37h035aef0_1
  - defaults/linux-64::dask==0.19.1=py37_0
  - defaults/linux-64::datashape==0.5.4=py37_1
  - defaults/linux-64::mkl-service==1.1.2=py37h90e4bf4_5
  - defaults/linux-64::numba==0.39.0=py37h04863e7_0
  - defaults/linux-64::numexpr==2.6.8=py37hd89afb7_0
  - defaults/linux-64::odo==0.5.1=py37_0
  - defaults/linux-64::pytables==3.4.4=py37ha205bf6_0
  - defaults/linux-64::pytest-arraydiff==0.2=py37h39e3cac_0
  - defaults/linux-64::pytest-astropy==0.4.0=py37_0
  - defaults/linux-64::pytest-doctestplus==0.1.3=py37_0
  - defaults/linux-64::pywavelets==1.0.0=py37h

In [30]:
url="http://cocl.us/Geospatial_data/Geospatial_Coordinates.csv"
coordinates=pd.read_csv(url)
coordinates.columns = ['Postcode', 'Latitude', 'Longitude']
df1 = pd.merge(tar_data,coordinates, on="Postcode")


In [31]:
df1.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
1,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
2,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937
3,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
4,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418


In [37]:
df1.columns

Index(['Postcode', 'Borough', 'Neighborhood', 'Latitude', 'Longitude'], dtype='object')

# Creating Toronto and Scarborough map

In [33]:
# Use geopy library to get the latitude and longitude values of Toronto
address = 'Toronto'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto City are {}, {}.'.format(latitude, longitude))

/home/jupyterlab/conda/lib/python3.6/site-packages/ipykernel_launcher.py:4: DeprecationWarning: Using Nominatim with the default "geopy/1.20.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  after removing the cwd from sys.path.


The geograpical coordinate of Toronto City are 43.653963, -79.387207.


In [38]:
# create map of Toronto using latitude and longitude values
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(df1['Latitude'], df1['Longitude'], df1['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
    
map_Toronto

# Analyzing each neighborhood using FOURSQUARE location data

### Define Foursquare Credentials and Version

In [39]:
CLIENT_ID = 'NY1X4ZTFDKEJY13TWNTNBPSEBCC4AVAG3WLXZVP4YMMT4LRW' # your Foursquare ID
CLIENT_SECRET = 'FSEH2VNFCDNZ4ZZKITQU1SBFO5H0KQ0XKOG54KK4K2CZMM2T' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100
radius = 500

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: NY1X4ZTFDKEJY13TWNTNBPSEBCC4AVAG3WLXZVP4YMMT4LRW
CLIENT_SECRET:FSEH2VNFCDNZ4ZZKITQU1SBFO5H0KQ0XKOG54KK4K2CZMM2T


In [48]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)
# type your answer here

Toronto_venues = getNearbyVenues(names=df1['Neighborhood'],
                                   latitudes=df1['Latitude'],
                                   longitudes=df1['Longitude']
                                  )


Harbourfront, Regent Park
Rouge, Malvern
Ryerson, Garden District
Highland Creek, Rouge Hill, Port Union
St. James Town
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Woburn
Central Bay Street
Christie
Cedarbrae
Adelaide, King, Richmond
Dovercourt Village, Dufferin
Scarborough Village
Harbourfront East, Toronto Islands, Union Station
Little Portugal, Trinity
East Birchmount Park, Ionview, Kennedy Park
The Danforth West, Riverdale
Design Exchange, Toronto Dominion Centre
Brockton, Exhibition Place, Parkdale Village
Clairlea, Golden Mile, Oakridge
The Beaches West, India Bazaar
Commerce Court, Victoria Hotel
Cliffcrest, Cliffside, Scarborough Village West
Studio District
Birch Cliff, Cliffside West
Lawrence Park
Roselawn
Dorset Park, Scarborough Town Centre, Wexford Heights
Davisville North
Forest Hill North, Forest Hill West
High Park, The Junction South
Maryvale, Wexford
North Toronto West
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Agincourt
Davisville
Harbor

In [49]:
Toronto_venues.columns

Index(['Neighborhood', 'Neighborhood Latitude', 'Neighborhood Longitude',
       'Venue', 'Venue Latitude', 'Venue Longitude', 'Venue Category'],
      dtype='object')

In [50]:
Toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Harbourfront, Regent Park",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Harbourfront, Regent Park",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Harbourfront, Regent Park",43.65426,-79.360636,Toronto Cooper Koo Family Cherry St YMCA Centre,43.653191,-79.357947,Gym / Fitness Center
3,"Harbourfront, Regent Park",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,"Harbourfront, Regent Park",43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot


In [51]:
# Check Count of Venues for Each Neighbourhood
Toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Agincourt,4,4,4,4,4,4
"Agincourt North, L'Amoreaux East, Milliken, Steeles East",3,3,3,3,3,3
Berczy Park,55,55,55,55,55,55
"Birch Cliff, Cliffside West",4,4,4,4,4,4
"Brockton, Exhibition Place, Parkdale Village",22,22,22,22,22,22
Business Reply Mail Processing Centre 969 Eastern,19,19,19,19,19,19
"CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara",16,16,16,16,16,16
"Cabbagetown, St. James Town",46,46,46,46,46,46


In [52]:
# Get the List of Unique Categories
print('There are {} neighbourhoods and {} uniques categories.'.format(Toronto_venues['Neighborhood'].count(),len(Toronto_venues['Venue Category'].unique())))

There are 1789 neighbourhoods and 250 uniques categories.


In [53]:
# one hot encoding
venues_onehot = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
venues_onehot['Neighborhood'] = Toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [venues_onehot.columns[-1]] + list(venues_onehot.columns[:-1])
#fixed_columns
venues_onehot = venues_onehot[fixed_columns]

venues_onehot.head()

,Yoga Studio,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Station,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Museum,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Post Office,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tech Startup,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Harbourfront, Regent Park",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [54]:
Toronto_grouped = venues_onehot.groupby('Neighborhood').mean().reset_index()
Toronto_grouped

,Neighborhood,Yoga Studio,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Station,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Museum,Music Store,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Post Office,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tech Startup,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store
0,"Adelaide, King, Richmond",0.000000,0.000000,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.040000,0.000000,0.00,0.010000,0.010000,0.000000,0.020000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.030000,0.000000,0.040000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0000,0.010000,0.000000,0.01,0.020000,0.000000,0.000000,0.030000,0.010000,0.000000,0.0,0.000000,0.050000,0.000000,0.00,0

In [55]:
Toronto_grouped.shape

(54, 250)

# List frequency of top 5 venues in each neighbourhood

In [56]:
num_top_venues = 5

for hood in Toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = Toronto_grouped[Toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
                 venue  freq
0                 Café  0.05
1          Coffee Shop  0.05
2  American Restaurant  0.04
3                  Bar  0.04
4           Steakhouse  0.04


----Agincourt----
                venue  freq
0      Sandwich Place  0.25
1      Breakfast Spot  0.25
2              Lounge  0.25
3  Chinese Restaurant  0.25
4        Noodle House  0.00


----Agincourt North, L'Amoreaux East, Milliken, Steeles East----
              venue  freq
0              Park  0.33
1        Playground  0.33
2  Asian Restaurant  0.33
3       Yoga Studio  0.00
4       Music Store  0.00


----Berczy Park----
                venue  freq
0         Coffee Shop  0.09
1        Cocktail Bar  0.05
2  Italian Restaurant  0.04
3         Cheese Shop  0.04
4              Bakery  0.04


----Birch Cliff, Cliffside West----
                   venue  freq
0  General Entertainment  0.25
1        College Stadium  0.25
2           Skating Rink  0.25
3                   Café  0.25

In [57]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

# Top 10 venues of each neighbourhood

In [83]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Toronto_grouped['Neighborhood']

for ind in np.arange(Toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Steakhouse,Bar,American Restaurant,Burger Joint,Bakery,Gym,Cosmetics Shop,Thai Restaurant
1,Agincourt,Breakfast Spot,Lounge,Chinese Restaurant,Sandwich Place,Women's Store,Discount Store,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant
2,"Agincourt North, L'Amoreaux East, Milliken, St...",Asian Restaurant,Park,Playground,Concert Hall,Diner,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant
3,Berczy Park,Coffee Shop,Cocktail Bar,Farmers Market,Seafood Restaurant,Café,Cheese Shop,Italian Restaurant,Beer Bar,Steakhouse,Bakery
4,"Birch Cliff, Cliffside West",College Stadium,Café,Skating Rink,General Entertainment,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store


In [78]:
neighborhoods_venues_sorted.shape

(54, 11)

In [84]:
#neighborhoods_venues_sorted['Neighbourhood']=neighborhoods_venues_sorted['Neighborhood']
#neighborhoods_venues_sorted.drop('Neighborhood',1,inplace=True)
neighborhoods_venues_sorted.columns

Index(['Neighborhood', '1st Most Common Venue', '2nd Most Common Venue',
       '3rd Most Common Venue', '4th Most Common Venue',
       '5th Most Common Venue', '6th Most Common Venue',
       '7th Most Common Venue', '8th Most Common Venue',
       '9th Most Common Venue', '10th Most Common Venue'],
      dtype='object')

# Segmenting and Clustering neighborhoods in Toronto

In [68]:
# set number of clusters
kclusters = 5

Toronto_grouped_clustering = Toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_ = np.append(kmeans.labels_,[2])
kmeans.labels_

array([2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 0, 1, 2, 2,
       2, 4, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [89]:
#Dataframe to include Clusters

Toronto_merged = df1
Toronto_merged['Cluster Labels'] = kmeans.labels_

Toronto_merged.columns
#neighborhoods_venues_sorted.columns
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Toronto_merged = Toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

Toronto_merged = Toronto_merged.dropna()
Toronto_merged.head(30)

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636,2,Coffee Shop,Park,Bakery,Pub,Breakfast Spot,Café,Restaurant,Mexican Restaurant,Theater,Yoga Studio
1,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,2,Fast Food Restaurant,Print Shop,Women's Store,Dessert Shop,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant
2,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937,0,Coffee Shop,Clothing Store,Cosmetics Shop,Café,Middle Eastern Restaurant,Theater,Fast Food Restaurant,Pizza Place,Japanese Restaurant,Ramen Restaurant
3,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,2,Construction & Landscaping,Bar,Women's Store,Discount Store,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant
4,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,2,Café,Coffee Shop,Restaurant,Hotel,Bakery,Clothing Store,Gastropub,Cosmetics Shop,Breakfast Spot,Cocktail Bar
5,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,2,Pizza Place,Tech Startup,Spa,Medical Center,Breakfast Spot,Electronics Store,Mexican Restaurant,Intersection,Rental Car Location,Creperie
6,M4E,East Toronto,The Beaches,43.676357,-79.293031,2,Health Food Store,Other Great Outdoors,Pub,Trail,Event Space,Ethiopian Restaurant,Electronics Store,Falafel Restaurant,Dessert Shop,Eastern European Restaurant
7,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,2,Coffee Shop,Cocktail Bar,Farmers Market,Seafood Restaurant,Café,Cheese Shop,Italian Restaurant,Beer Bar,Steakhouse,Bakery
8,M1G,Scarborough,Woburn,43.770992,-79.216917,2,Coffee Shop,Korean Restaurant,Convenience Store,Women's Store,Electronics Store,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Event Space
9,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383,2,Coffee Shop,Italian Restaurant,Café,Middle Eastern Restaurant,Burger Joint,Sandwich Place,Bubble Tea Shop,Restaurant,Salad Place,Japanese Restaurant


# Map of Cluster of neighborhoods

In [91]:
# Create Map

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Toronto_merged['Latitude'], Toronto_merged['Longitude'], Toronto_merged['Neighborhood'], Toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# Examine Clusters
Examine each cluster to determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, we can then assign a name to each cluster.

### Cluster 0

In [92]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 0]

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937,0,Coffee Shop,Clothing Store,Cosmetics Shop,Café,Middle Eastern Restaurant,Theater,Fast Food Restaurant,Pizza Place,Japanese Restaurant,Ramen Restaurant
37,M1S,Scarborough,Agincourt,43.794200,-79.262029,0,Breakfast Spot,Lounge,Chinese Restaurant,Sandwich Place,Women's Store,Discount Store,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant
40,M6S,West Toronto,"Runnymede, Swansea",43.651571,-79.484450,0,Pizza Place,Coffee Shop,Café,Sushi Restaurant,Italian Restaurant,Diner,Bookstore,Boutique,Sandwich Place,Burrito Place


### Cluster 1

In [93]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 1]

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
41,M1T,Scarborough,"Clarks Corners, Sullivan, Tam O'Shanter",43.781638,-79.304302,1,Pizza Place,Pharmacy,Noodle House,Bank,Fried Chicken Joint,Fast Food Restaurant,Chinese Restaurant,Thai Restaurant,Italian Restaurant,Creperie


### Cluster 2

In [97]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 2]

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636,2,Coffee Shop,Park,Bakery,Pub,Breakfast Spot,Café,Restaurant,Mexican Restaurant,Theater,Yoga Studio
1,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,2,Fast Food Restaurant,Print Shop,Women's Store,Dessert Shop,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant
3,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,2,Construction & Landscaping,Bar,Women's Store,Discount Store,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant
4,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,2,Café,Coffee Shop,Restaurant,Hotel,Bakery,Clothing Store,Gastropub,Cosmetics Shop,Breakfast Spot,Cocktail Bar
5,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,2,Pizza Place,Tech Startup,Spa,Medical Center,Breakfast Spot,Electronics Store,Mexican Restaurant,Intersection,Rental Car Location,Creperie
6,M4E,East Toronto,The Beaches,43.676357,-79.293031,2,Health Food Store,Other Great Outdoors,Pub,Trail,Event Space,Ethiopian Restaurant,Electronics Store,Falafel Restaurant,Dessert Shop,Eastern European Restaurant
7,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,2,Coffee Shop,Cocktail Bar,Farmers Market,Seafood Restaurant,Café,Cheese Shop,Italian Restaurant,Beer Bar,Steakhouse,Bakery
8,M1G,Scarborough,Woburn,43.770992,-79.216917,2,Coffee Shop,Korean Restaurant,Convenience Store,Women's Store,Electronics Store,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Event Space
9,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383,2,Coffee Shop,Italian Restaurant,Café,Middle Eastern Restaurant,Burger Joint,Sandwich Place,Bubble Tea Shop,Restaurant,Salad Place,Japanese Restaurant
10,M6G,Downtown Toronto,Christie,43.669542,-79.422564,2,Café,Grocery Store,Park,Restaurant,Coffee Shop,Italian Restaurant,Nightclub,Diner,Baby Store,Convenience Store


In [98]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 3]

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
16,M6J,West Toronto,"Little Portugal, Trinity",43.647927,-79.41975,3,Bar,Coffee Shop,Men's Store,Asian Restaurant,Pizza Place,New American Restaurant,Cocktail Bar,Café,Bakery,Restaurant


In [99]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 4]

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
45,M4V,Central Toronto,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",43.686412,-79.400049,4,Pub,Coffee Shop,Liquor Store,Light Rail Station,Fried Chicken Joint,Supermarket,Bagel Shop,Sports Bar,American Restaurant,Pizza Place


# Find which cluster does Central Bay Street belong to and find which neighbourhoods in the cluster are from Scarborough

In [101]:
Toronto_merged[Toronto_merged['Neighborhood']=='Central Bay Street']

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383,2,Coffee Shop,Italian Restaurant,Café,Middle Eastern Restaurant,Burger Joint,Sandwich Place,Bubble Tea Shop,Restaurant,Salad Place,Japanese Restaurant


In [102]:
neigh_candidates = Toronto_merged.loc[(Toronto_merged['Cluster Labels'] == 2) & (Toronto_merged['Borough'] =='Scarborough')]

In [103]:
neigh_candidates.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,2,Fast Food Restaurant,Print Shop,Women's Store,Dessert Shop,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant
3,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,2,Construction & Landscaping,Bar,Women's Store,Discount Store,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant
5,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,2,Pizza Place,Tech Startup,Spa,Medical Center,Breakfast Spot,Electronics Store,Mexican Restaurant,Intersection,Rental Car Location,Creperie
8,M1G,Scarborough,Woburn,43.770992,-79.216917,2,Coffee Shop,Korean Restaurant,Convenience Store,Women's Store,Electronics Store,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Event Space
11,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,2,Athletics & Sports,Caribbean Restaurant,Hakka Restaurant,Fried Chicken Joint,Bakery,Lounge,Thai Restaurant,Bank,Dumpling Restaurant,Donut Shop


# Creating map of selected neighborhoods

In [105]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(neigh_candidates['Latitude'], neigh_candidates['Longitude'], neigh_candidates['Neighborhood'], neigh_candidates['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [106]:
neigh_candidates.reset_index(drop=True,inplace=True)

In [107]:
neigh_candidates

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,2,Fast Food Restaurant,Print Shop,Women's Store,Dessert Shop,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,2,Construction & Landscaping,Bar,Women's Store,Discount Store,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,2,Pizza Place,Tech Startup,Spa,Medical Center,Breakfast Spot,Electronics Store,Mexican Restaurant,Intersection,Rental Car Location,Creperie
3,M1G,Scarborough,Woburn,43.770992,-79.216917,2,Coffee Shop,Korean Restaurant,Convenience Store,Women's Store,Electronics Store,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Event Space
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,2,Athletics & Sports,Caribbean Restaurant,Hakka Restaurant,Fried Chicken Joint,Bakery,Lounge,Thai Restaurant,Bank,Dumpling Restaurant,Donut Shop
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476,2,Playground,Women's Store,Diner,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029,2,Playground,Coffee Shop,Department Store,Discount Store,Farmers Market,Falafel Restaurant,Fast Food Restaurant,Event Space,Diner,Electronics Store
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577,2,Bus Line,Bakery,Soccer Field,Fast Food Restaurant,Metro Station,Park,Bus Station,Intersection,Dumpling Restaurant,Donut Shop
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476,2,Motel,American Restaurant,Women's Store,Discount Store,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848,2,College Stadium,Café,Skating Rink,General Entertainment,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store


In [108]:
neigh_candidates.columns

Index(['Postcode', 'Borough', 'Neighborhood', 'Latitude', 'Longitude',
       'Cluster Labels', '1st Most Common Venue', '2nd Most Common Venue',
       '3rd Most Common Venue', '4th Most Common Venue',
       '5th Most Common Venue', '6th Most Common Venue',
       '7th Most Common Venue', '8th Most Common Venue',
       '9th Most Common Venue', '10th Most Common Venue'],
      dtype='object')

# Looking for Indian restaurants and grocery store

In [109]:
neigh_candidates[neigh_candidates[['1st Most Common Venue', '2nd Most Common Venue',
       '3rd Most Common Venue', '4th Most Common Venue',
       '5th Most Common Venue', '6th Most Common Venue',
       '7th Most Common Venue', '8th Most Common Venue',
       '9th Most Common Venue', '10th Most Common Venue']] == 'Grocery Store']

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [110]:
neigh_candidates[neigh_candidates[['1st Most Common Venue', '2nd Most Common Venue',
       '3rd Most Common Venue', '4th Most Common Venue',
       '5th Most Common Venue', '6th Most Common Venue',
       '7th Most Common Venue', '8th Most Common Venue',
       '9th Most Common Venue', '10th Most Common Venue']] == 'Indian Restaurant']

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# looking for grocery stores in neighbourhood id 10

In [111]:
search_query = 'Grocery Store'
radius = 10000
LIMIT = 10
latitude = neigh_candidates.loc[10,'Latitude']
longitude = neigh_candidates.loc[10,'Longitude']
print("latitude is {},longitude is {}".format(latitude,longitude))

latitude is 43.7574096,longitude is -79.27330400000001


# Define the corresponding URL

In [112]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=NY1X4ZTFDKEJY13TWNTNBPSEBCC4AVAG3WLXZVP4YMMT4LRW&client_secret=FSEH2VNFCDNZ4ZZKITQU1SBFO5H0KQ0XKOG54KK4K2CZMM2T&ll=43.7574096,-79.27330400000001&v=20180605&query=Grocery Store&radius=10000&limit=10'

# Send the get request and obtain results

In [113]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d1388109df1dc002f4a9719'},
 'response': {'venues': [{'id': '4bfebef9e584c928402b6d25',
    'name': 'Salvation Army Thrift Store',
    'location': {'address': '1225 Kennedy Rd',
     'lat': 43.755782105509475,
     'lng': -79.2762077195083,
     'labeledLatLngs': [{'label': 'display',
       'lat': 43.755782105509475,
       'lng': -79.2762077195083}],
     'distance': 295,
     'postalCode': 'M1P 2L4',
     'cc': 'CA',
     'city': 'Scarborough',
     'state': 'ON',
     'country': 'Canada',
     'formattedAddress': ['1225 Kennedy Rd',
      'Scarborough ON M1P 2L4',
      'Canada']},
    'categories': [{'id': '4bf58dd8d48988d101951735',
      'name': 'Thrift / Vintage Store',
      'pluralName': 'Thrift / Vintage Stores',
      'shortName': 'Thrift / Vintage',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/default_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1561561104',
    'hasPerk': False},


# Get relevant part of JSON and transform it into pandas dataframe

In [114]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)

In [115]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Salvation Army Thrift Store,Thrift / Vintage Store,1225 Kennedy Rd,CA,Scarborough,Canada,NaN,295,"[1225 Kennedy Rd, Scarborough ON M1P 2L4, Canada]","[{'label': 'display', 'lat': 43.75578210550947...",43.755782,-79.276208,NaN,M1P 2L4,ON,4bfebef9e584c928402b6d25
1,Sherwin-Williams Commercial Paint Store,Hardware Store,260 Nantucket Blvd,CA,Scarborough,Canada,NaN,432,"[260 Nantucket Blvd, Scarborough ON M1P 2P4, C...","[{'label': 'display', 'lat': 43.76047864, 'lng...",43.760479,-79.276595,NaN,M1P 2P4,ON,57cf339c498eef8d7dfa0251
2,Disney Store,Toy / Game Store,300 Borough Drive,CA,Scarborough,Canada,in Scarborough Town Centre,2413,[300 Borough Drive (in Scarborough Town Centre...,"[{'label': 'display', 'lat': 43.775537, 'lng':...",43.775537,-79.256833,NaN,M1P 4P5,ON,5085ec39e4b0b1ead2eb0818
3,TELUS Store,IT Services,300 Borough Dr UNIT 95,CA,Scarborough,Canada,NaN,2382,"[300 Borough Dr UNIT 95, Scarborough ON M1P 4...","[{'label': 'display', 'lat': 43.77564497500000...",43.775645,-79.257800,NaN,M1P 4P5,ON,4bc33a9f4cdfc9b626029721
4,spirit Halloween Store,Miscellaneous Shop,1333 Kennedy Road Scarborough,CA,NaN,Canada,NaN,668,"[1333 Kennedy Road Scarborough, Canada]","[{'label': 'display', 'lat': 43.76163979192874...",43.761640,-79.279213,NaN,NaN,NaN,524852cd11d2e58c893f7088
5,Pack and Store,Storage Facility,1399 Kennedy Road,CA,Scarborough,Canada,kennedy and ellesmere,720,"[1399 Kennedy Road (kennedy and ellesmere), Sc...","[{'label': 'display', 'lat': 43.76287460327148...",43.762875,-79.278107,NaN,NaN,ON,546ba557498ede4d7a37aa7f
6,Ghadir Mid-Eastern Grocery,Grocery Store,1821 Lawrence Ave E,CA,Toronto,Canada,Pharmacy Ave,2921,"[1821 Lawrence Ave E (Pharmacy Ave), Toronto O...","[{'label': 'display', 'lat': 43.74363845933303...",43.743638,-79.304233,NaN,M1R 2Y3,ON,4dd19e051838a75196486487
7,Pooja Store,Miscellaneous Shop,1209 Ellesmere Rd,CA,Scarborough,Canada,Midland Ave,1339,"[1209 Ellesmere Rd (Midland Ave), Scarborough ...","[{'label': 'display', 'lat': 43.76885821311027...",43.768858,-79.268171,NaN,NaN,ON,4eb6ad1d722e578a4ff973a4
8,Tommy Hilfiger Company Store,Clothing Store,300 Borough Dr,CA,Scarborough,Canada,NaN,2435,"[300 Borough Dr, Scarborough ON M1P 4P5, Canada]","[{'label': 'display', 'lat': 43.77601546103458...",43.776015,-79.257369,Scarborough City Centre,M1P 4P5,ON,4d1127d65f3376eb86661386
9,Everything for a Dollar Store,Miscellaneous Shop,2650 Lawrence Ave E,CA,Scarborough,Canada,NaN,931,"[2650 Lawrence Ave E, Scarborough ON, Canada]","[{'label': 'display', 'lat': 43.7523719727376,...",43.752372,-79.264047,NaN,NaN,ON,4ec96e336c251306cdc4732a


# Clean the data to a table containing grocery store 

In [116]:
dataframe_filtered = dataframe_filtered[(dataframe_filtered['categories']=="Market") | (dataframe_filtered['categories']==
                                       "Convenience Store") | (dataframe_filtered['categories']=="Grocery Store")]


In [117]:
dataframe_filtered.reset_index(drop=True,inplace=True)

In [118]:
dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Ghadir Mid-Eastern Grocery,Grocery Store,1821 Lawrence Ave E,CA,Toronto,Canada,Pharmacy Ave,2921,"[1821 Lawrence Ave E (Pharmacy Ave), Toronto O...","[{'label': 'display', 'lat': 43.74363845933303...",43.743638,-79.304233,NaN,M1R 2Y3,ON,4dd19e051838a75196486487


# Ratings of the Grocery store

In [119]:
def rate(id):
    venue_id = id
    url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
    result = requests.get(url).json()
    try:
        print(result['response']['venue']['rating'])
    except:
        print('This venue has not been rated yet.')


In [120]:
dataframe_filtered['id'].apply(lambda id: rate(id) )

8.2


0    None
Name: id, dtype: object

## Final Candidates

In [121]:
final_candidate = neigh_candidates.drop([0,1,2,3,4,5,6,7,8,9,11,12,13])

In [122]:
final_candidate

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,M1P,Scarborough,"Dorset Park, Scarborough Town Centre, Wexford ...",43.75741,-79.273304,2,Indian Restaurant,Vietnamese Restaurant,Pet Store,Chinese Restaurant,Latin American Restaurant,Women's Store,Dumpling Restaurant,Dog Run,Doner Restaurant,Donut Shop


### Searching for Indian Restaurants at id 10

In [123]:
latitude = final_candidate.loc[10,'Latitude']
longitude = final_candidate.loc[10,'Longitude']
print("latitude is {},longitude is {}".format(latitude,longitude))

search_query = 'Indian Restaurant'
radius = 10000
print(search_query + ' .... OK!')

latitude is 43.7574096,longitude is -79.27330400000001
Indian Restaurant .... OK!


In [124]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=NY1X4ZTFDKEJY13TWNTNBPSEBCC4AVAG3WLXZVP4YMMT4LRW&client_secret=FSEH2VNFCDNZ4ZZKITQU1SBFO5H0KQ0XKOG54KK4K2CZMM2T&ll=43.7574096,-79.27330400000001&v=20180605&query=Indian Restaurant&radius=10000&limit=10'

In [125]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d138eb6348eba0023b31725'},
 'response': {'venues': [{'id': '4bf96c435317a593a23a017f',
    'name': 'Karaikudi Chettinad South Indian Restaurant',
    'location': {'address': '1225 Kennedy Rd',
     'crossStreet': 'at Forbes (Between Lawrence and Ellesmere)',
     'lat': 43.756041539453136,
     'lng': -79.27627591942013,
     'labeledLatLngs': [{'label': 'display',
       'lat': 43.756041539453136,
       'lng': -79.27627591942013}],
     'distance': 283,
     'postalCode': 'M1P 4Y1',
     'cc': 'CA',
     'city': 'Toronto',
     'state': 'ON',
     'country': 'Canada',
     'formattedAddress': ['1225 Kennedy Rd (at Forbes (Between Lawrence and Ellesmere))',
      'Toronto ON M1P 4Y1',
      'Canada']},
    'categories': [{'id': '4bf58dd8d48988d10f941735',
      'name': 'Indian Restaurant',
      'pluralName': 'Indian Restaurants',
      'shortName': 'Indian',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/indian_',
       's

In [126]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '4bf58dd8d48988d10f941735', 'name': 'I...",False,4bf96c435317a593a23a017f,1225 Kennedy Rd,CA,Toronto,Canada,at Forbes (Between Lawrence and Ellesmere),283,[1225 Kennedy Rd (at Forbes (Between Lawrence ...,"[{'label': 'display', 'lat': 43.75604153945313...",43.756042,-79.276276,NaN,M1P 4Y1,ON,Karaikudi Chettinad South Indian Restaurant,v-1561562807,33638214
1,"[{'id': '4bf58dd8d48988d10f941735', 'name': 'I...",False,5b89816add70c5002cc469c4,925 Warden Ave,CA,Toronto,Canada,NaN,3201,"[925 Warden Ave, Toronto ON M1L 4C5, Canada]","[{'label': 'display', 'lat': 43.730846, 'lng':...",43.730846,-79.288550,NaN,M1L 4C5,ON,Barwachi Indian Restaurant,v-1561562807,NaN
2,"[{'id': '4bf58dd8d48988d10f941735', 'name': 'I...",False,4d570727fb65236a7f600db4,NaN,CA,Toronto,Canada,NaN,4231,"[Toronto ON, Canada]","[{'label': 'display', 'lat': 43.79192991300102...",43.791930,-79.251253,NaN,NaN,ON,Silver Spoon Pak-Indian Restaurant,v-1561562807,NaN
3,"[{'id': '4bf58dd8d48988d10f941735', 'name': 'I...",False,4b93d4a7f964a520eb5334e3,"1571 Sandhurst Cir, Unit 153",CA,Toronto,Canada,McCowan Rd (Woodside Square),5876,"[1571 Sandhurst Cir, Unit 153 (McCowan Rd (Woo...","[{'label': 'display', 'lat': 43.81011674776289...",43.810117,-79.269275,NaN,M1V 1V2,ON,Saravanaa Bhavan South Indian Restaurant,v-1561562807,NaN
4,"[{'id': '4bf58dd8d48988d10f941735', 'name': 'I...",False,5c7da4b08ad62e0039395a08,1646 Victoria Park Ave,CA,North York,Canada,NaN,3937,"[1646 Victoria Park Ave, North York ON M1R 1P7...","[{'label': 'display', 'lat': 43.7309565, 'lng'...",43.730956,-79.305799,NaN,M1R 1P7,ON,Blue Water Curry & Roti West Indian Restaurant,v-1561562807,NaN


In [127]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Karaikudi Chettinad South Indian Restaurant,Indian Restaurant,1225 Kennedy Rd,CA,Toronto,Canada,at Forbes (Between Lawrence and Ellesmere),283,[1225 Kennedy Rd (at Forbes (Between Lawrence ...,"[{'label': 'display', 'lat': 43.75604153945313...",43.756042,-79.276276,NaN,M1P 4Y1,ON,4bf96c435317a593a23a017f
1,Barwachi Indian Restaurant,Indian Restaurant,925 Warden Ave,CA,Toronto,Canada,NaN,3201,"[925 Warden Ave, Toronto ON M1L 4C5, Canada]","[{'label': 'display', 'lat': 43.730846, 'lng':...",43.730846,-79.288550,NaN,M1L 4C5,ON,5b89816add70c5002cc469c4
2,Silver Spoon Pak-Indian Restaurant,Indian Restaurant,NaN,CA,Toronto,Canada,NaN,4231,"[Toronto ON, Canada]","[{'label': 'display', 'lat': 43.79192991300102...",43.791930,-79.251253,NaN,NaN,ON,4d570727fb65236a7f600db4
3,Saravanaa Bhavan South Indian Restaurant,Indian Restaurant,"1571 Sandhurst Cir, Unit 153",CA,Toronto,Canada,McCowan Rd (Woodside Square),5876,"[1571 Sandhurst Cir, Unit 153 (McCowan Rd (Woo...","[{'label': 'display', 'lat': 43.81011674776289...",43.810117,-79.269275,NaN,M1V 1V2,ON,4b93d4a7f964a520eb5334e3
4,Blue Water Curry & Roti West Indian Restaurant,Indian Restaurant,1646 Victoria Park Ave,CA,North York,Canada,NaN,3937,"[1646 Victoria Park Ave, North York ON M1R 1P7...","[{'label': 'display', 'lat': 43.7309565, 'lng'...",43.730956,-79.305799,NaN,M1R 1P7,ON,5c7da4b08ad62e0039395a08
5,Patio Indian Restaurant,Indian Restaurant,15 Gervais Dr.,CA,Toronto,Canada,across superstore,6371,"[15 Gervais Dr. (across superstore), Toronto O...","[{'label': 'display', 'lat': 43.722103, 'lng':...",43.722103,-79.335655,NaN,M3C 1Y8,ON,59e94c0260255e613b025e38
6,The Famous Indian Restaurant,Indian Restaurant,1410 Gerrard St E,CA,Toronto,Canada,at Ashdale Ave,10246,"[1410 Gerrard St E (at Ashdale Ave), Toronto O...","[{'label': 'display', 'lat': 43.672339, 'lng':...",43.672339,-79.321941,"Little India, Toronto, ON",M4L 1Z2,ON,4ae4c793f964a5201b9e21e3
7,Earth Indian Restaurant,Indian Restaurant,6036 Yonge Street,CA,North York,Canada,Cummer Ave,12063,"[6036 Yonge Street (Cummer Ave), North York ON...","[{'label': 'display', 'lat': 43.78752401559961...",43.787524,-79.417466,NaN,M2M 3W5,ON,4b43dde2f964a52099ec25e3
8,Indian Grocerries and Spice,Market,1983 Lawrence Ave E,CA,Toronto,Canada,Lilian Dr,2245,"[1983 Lawrence Ave E (Lilian Dr), Toronto ON M...","[{'label': 'display', 'lat': 43.74530493763506...",43.745305,-79.295647,NaN,M1R 2Z1,ON,4e204c5118501a60d2fe2bf4
9,Kim Ling Indian Chinese Restaurant,Indian Chinese Restaurant,2173 Lawrence Ave. East,CA,Toronto,Canada,at Birchmount Rd,1407,"[2173 Lawrence Ave. East (at Birchmount Rd), T...","[{'label': 'display', 'lat': 43.74740094493909...",43.747401,-79.284002,NaN,NaN,ON,4bdb095ec79cc92877c980e9


In [128]:
dataframe_filtered['id'].apply(lambda id: rate(id) )

6.3
This venue has not been rated yet.
7.5
7.6
This venue has not been rated yet.
This venue has not been rated yet.
This venue has not been rated yet.
5.9
This venue has not been rated yet.
This venue has not been rated yet.


0    None
1    None
2    None
3    None
4    None
5    None
6    None
7    None
8    None
9    None
Name: id, dtype: object

In [129]:
address = 'Scarborough, ON'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

/home/jupyterlab/conda/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.20.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  This is separate from the ipykernel package so we can avoid doing imports until


43.773077 -79.257774


In [131]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around scarborough


# add the neighbourhoods as blue circle markers
for lat, lng, label in zip(final_candidate.Latitude, final_candidate.Longitude, final_candidate.Neighborhood):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

## This is the ideal place according to the requirements mentioned.